In [ ]:
# Example modified from https://github.com/samuelpeet/flashgamma/blob/c00f388856cd3651b706a7b117ff90930aa5ee54/examples/eg3_gamma_surface.py
# Source is licensed under the MIT license
# Original author is Samuel Peet

In [ ]:
!pip install flashgamma pymedphys

In [ ]:
import urllib.request

import numpy as np
import matplotlib.pyplot as plt

import flashgamma
import pymedphys

In [ ]:
file_details = {
    'snc': {
        'url': 'https://raw.githubusercontent.com/samuelpeet/flashgamma/c00f388856cd3651b706a7b117ff90930aa5ee54/examples/sample_snc_extracted_data.snc',
        'filetype': 'snc_extracted'
    },
    'arccheck': {
        'url': 'https://raw.githubusercontent.com/samuelpeet/flashgamma/master/examples/sample_arccheck_measurement.txt',
        'filetype': 'arccheck'
    }
}

In [ ]:
for key, item in file_details.items():
    url = item['url']
    
    filename = url.split('/')[-1]
    item['filename'] = filename
    
    urllib.request.urlretrieve(url, filename)

In [ ]:
file_details

In [ ]:
distributions = {
    key: flashgamma.load_file(item['filename'], item['filetype'])
    for key, item in file_details.items()
}

ref_dist = distributions['arccheck']
eval_dist = distributions['snc']

ref_xx, ref_yy = ref_dist.position
eval_xx, eval_yy = eval_dist.position

ref_dist_numpy = ref_dist.data.copy()
eval_dist_numpy = eval_dist.data.copy()

In [ ]:
coords_ref = (ref_yy[:,0], ref_xx[0,:])
coords_eval = (eval_yy[:,0], eval_xx[0,:])

In [ ]:
np.shape(ref_dist_numpy)

In [ ]:
np.shape(eval_dist_numpy)

In [ ]:
eval_dist = eval_dist.scale_grid(
    reference_distribution=ref_dist, new_resolution=3
)

In [ ]:
np.shape(eval_dist.data)

In [ ]:
np.shape(ref_dist.data)

In [ ]:
dose_criteria = np.arange(0.1, 3.01, 0.01)  # %
distance_criteria = np.arange(0.1, 2.51, 0.01)  # mm

In [ ]:
lower_percent_dose_cutoff = 20
local = False

In [ ]:
pymedphys_gamma_options = {
    'dose_percent_threshold': dose_criteria,
    'distance_mm_threshold': distance_criteria,
    'lower_percent_dose_cutoff': lower_percent_dose_cutoff,
    'interp_fraction': 10,  # Should be 10 or more for more accurate results
    'max_gamma': 1.1,
    'random_subset': None,
    'local_gamma': local,
    'ram_available': 2**32,  # 4 GB
    'quiet': False
}

In [ ]:
pass_rates = flashgamma.gamma_pass_rate(
    ref_dist,
    eval_dist,
    delta_dose=dose_criteria,
    delta_distance=distance_criteria,
    threshold=lower_percent_dose_cutoff,
    local=local
)

In [ ]:
np.shape(pass_rates)

In [ ]:
# pass_rates

In [ ]:
def plot_passing_rates(passing_rates):
    f = plt.figure()
    ax1 = plt.gca()

    xx, yy = np.meshgrid(distance_criteria, dose_criteria, )
    im1 = ax1.pcolormesh(xx, yy, passing_rates, shading='nearest')

    co1 = ax1.contour(distance_criteria, dose_criteria, passing_rates, colors='k', linewidth=1)
    zc = co1.collections
    plt.setp(zc, linewidth=1)
    plt.clabel(co1, fontsize=10, fmt='%1.0f')

    co2 = ax1.contour(distance_criteria, dose_criteria, passing_rates, colors='#f70c0c', levels=[95])
    zc = co2.collections
    plt.setp(zc, linewidth=2)
    plt.clabel(co2, fontsize=12, fmt='%1.0f')

    ax1.set_title('Gamma Pass Rates')
    ax1.set_xlabel('Distance to agreement (mm)')
    ax1.set_ylabel('Dose difference (%)')
    c1 = plt.colorbar(im1, ax=ax1)
    c1.set_label('Pass rate (%)', rotation=270, labelpad=12)

## FlashGamma's pass rates

In [ ]:
plot_passing_rates(pass_rates)

In [ ]:
pymedphys_gamma = pymedphys.gamma(
    coords_ref, ref_dist_numpy,
    coords_eval, eval_dist_numpy,
    **pymedphys_gamma_options)

In [ ]:
pymedphys_pass_rates = np.zeros_like(pass_rates)
for key, item in pymedphys_gamma.items():
    i = list(dose_criteria).index(key[0])
    j = list(distance_criteria).index(key[1])
    
    valid_gamma = item[~np.isnan(item)]
    passing = 100 * np.sum(valid_gamma <= 1) / len(valid_gamma)
    pymedphys_pass_rates[i, j] = passing

## PyMedPhys' pass rates

In [ ]:
plot_passing_rates(pymedphys_pass_rates)